# SQL functions
---

# SparkContext and SparkSession

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext(master = 'local')

from pyspark.sql import SparkSession
spark = SparkSession.builder \
            .appName("Python Spark SQL basic example") \
            .config("spark.some.config.option", "some-value") \
            .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/28 02:16:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Import data

In [2]:
iris = spark.read.csv('data/iris.csv', header=True, inferSchema=True)
iris.show(5)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [3]:
prostate = spark.read.csv('data/prostate.csv', header=True, inferSchema=True)
prostate.show(5)

+------------+-----------+---+------------+---+------------+-------+-----+------------+
|      lcavol|    lweight|age|        lbph|svi|         lcp|gleason|pgg45|        lpsa|
+------------+-----------+---+------------+---+------------+-------+-----+------------+
|-0.579818495|2.769458829| 50|-1.386294361|  0|-1.386294361|      6|    0|-0.430782916|
|-0.994252273|3.319625728| 58|-1.386294361|  0|-1.386294361|      6|    0|-0.162518929|
|-0.510825624|2.691243083| 74|-1.386294361|  0|-1.386294361|      7|   20|-0.162518929|
|-1.203972804|3.282789151| 58|-1.386294361|  0|-1.386294361|      6|    0|-0.162518929|
| 0.751416089|3.432372999| 62|-1.386294361|  0|-1.386294361|      6|    0| 0.371563556|
+------------+-----------+---+------------+---+------------+-------+-----+------------+
only showing top 5 rows



# Functions

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np
import pandas as pd

## abs

In [5]:
prostate.select('lpsa', abs(prostate.lpsa).alias('abs(lpsa)')).show(5)

+------------+-----------+
|        lpsa|  abs(lpsa)|
+------------+-----------+
|-0.430782916|0.430782916|
|-0.162518929|0.162518929|
|-0.162518929|0.162518929|
|-0.162518929|0.162518929|
| 0.371563556|0.371563556|
+------------+-----------+
only showing top 5 rows



## acos
arccosine

In [12]:
# generate 5 random negative values and 5 random positive values and combining into a list, create DataFrame
pdf = pd.DataFrame({
    'x': list(-np.random.rand(5)) + list(np.random.rand(5))
})
df = spark.createDataFrame(pdf)
df.show(5)

+--------------------+
|                   x|
+--------------------+
|-0.41093841088017535|
|  -0.779655582698541|
| -0.9536119404173016|
|-0.36862819054292817|
| -0.9941258873864552|
+--------------------+
only showing top 5 rows



In [13]:
df.select('x', acos(df.x)).show(5)

+--------------------+------------------+
|                   x|           ACOS(x)|
+--------------------+------------------+
|-0.41093841088017535|1.9942794900121013|
|  -0.779655582698541| 2.464911951423669|
| -0.9536119404173016| 2.835811014801651|
|-0.36862819054292817| 1.948329179619281|
| -0.9941258873864552|  3.03315029557948|
+--------------------+------------------+
only showing top 5 rows



## add_months

In [14]:
import datetime

In [15]:
base = datetime.date.today()          # get today's date
# generate a list of dates starting from base(today's date) with a range of 10 days, repeat 10 times
date_list = [base + datetime.timedelta(days=x) for x in list(range(0, 10))*10]
# create a Pandas DateFrame with the generated list of dates under the column 'dates'
pdf = pd.DataFrame({
    'dates': date_list
})
# convert the Pandas DataFrame into a PySpark DataFrame
df = spark.createDataFrame(pdf)
df.show(5)

+----------+
|     dates|
+----------+
|2025-02-28|
|2025-03-01|
|2025-03-02|
|2025-03-03|
|2025-03-04|
+----------+
only showing top 5 rows



In [16]:
# add 2 months to the dates
df.select('dates', add_months(df.dates, 2).alias('new_dates')).show(5)

+----------+----------+
|     dates| new_dates|
+----------+----------+
|2025-02-28|2025-04-28|
|2025-03-01|2025-05-01|
|2025-03-02|2025-05-02|
|2025-03-03|2025-05-03|
|2025-03-04|2025-05-04|
+----------+----------+
only showing top 5 rows



## approx_count_distinct
estimate the approximate number of distint values in a column

In [17]:
prostate.select(approx_count_distinct(prostate.gleason)).show(5)

25/02/28 02:26:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------------------+
|approx_count_distinct(gleason)|
+------------------------------+
|                             4|
+------------------------------+



In [18]:
iris.select(approx_count_distinct(iris.species)).show(5)

+------------------------------+
|approx_count_distinct(species)|
+------------------------------+
|                             3|
+------------------------------+



## array

In [19]:
iris.show(5)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [20]:
# create a new column that contains an array of the four feature columns
df_arr = iris.select('species', array(['sepal_length', 'sepal_width', 'petal_length', 'petal_width']).alias('features'))
df_arr.show(5)

+-------+--------------------+
|species|            features|
+-------+--------------------+
| setosa|[5.1, 3.5, 1.4, 0.2]|
| setosa|[4.9, 3.0, 1.4, 0.2]|
| setosa|[4.7, 3.2, 1.3, 0.2]|
| setosa|[4.6, 3.1, 1.5, 0.2]|
| setosa|[5.0, 3.6, 1.4, 0.2]|
+-------+--------------------+
only showing top 5 rows



## array_contains

In [22]:
# create a new column that holds a boolean value wheterh a specific value exists within the features array column
df = df_arr.select('species', 'features', array_contains(df_arr.features, 1.4).alias('new_features'))
df.show(5)

+-------+--------------------+------------+
|species|            features|new_features|
+-------+--------------------+------------+
| setosa|[5.1, 3.5, 1.4, 0.2]|        true|
| setosa|[4.9, 3.0, 1.4, 0.2]|        true|
| setosa|[4.7, 3.2, 1.3, 0.2]|       false|
| setosa|[4.6, 3.1, 1.5, 0.2]|       false|
| setosa|[5.0, 3.6, 1.4, 0.2]|        true|
+-------+--------------------+------------+
only showing top 5 rows



In [23]:
# filter the rows where the new_features column is True
df.filter(df.new_features).show(5)

+-------+--------------------+------------+
|species|            features|new_features|
+-------+--------------------+------------+
| setosa|[5.1, 3.5, 1.4, 0.2]|        true|
| setosa|[4.9, 3.0, 1.4, 0.2]|        true|
| setosa|[5.0, 3.6, 1.4, 0.2]|        true|
| setosa|[4.6, 3.4, 1.4, 0.3]|        true|
| setosa|[4.4, 2.9, 1.4, 0.2]|        true|
+-------+--------------------+------------+
only showing top 5 rows



## asc
asc returns a **sort expression**, which can be used as argument of sort functions 
such as pyspark.sql.DataFrame.sort and pyspark.sql.DataFrame.orderBy

In [24]:
# sort prostate by lpsa column in ascending order
prostate.sort(prostate.lpsa.asc()).show(5)

+------------+-----------+---+------------+---+------------+-------+-----+------------+
|      lcavol|    lweight|age|        lbph|svi|         lcp|gleason|pgg45|        lpsa|
+------------+-----------+---+------------+---+------------+-------+-----+------------+
|-0.579818495|2.769458829| 50|-1.386294361|  0|-1.386294361|      6|    0|-0.430782916|
|-0.994252273|3.319625728| 58|-1.386294361|  0|-1.386294361|      6|    0|-0.162518929|
|-0.510825624|2.691243083| 74|-1.386294361|  0|-1.386294361|      7|   20|-0.162518929|
|-1.203972804|3.282789151| 58|-1.386294361|  0|-1.386294361|      6|    0|-0.162518929|
| 0.751416089|3.432372999| 62|-1.386294361|  0|-1.386294361|      6|    0| 0.371563556|
+------------+-----------+---+------------+---+------------+-------+-----+------------+
only showing top 5 rows



In [25]:
prostate.orderBy(prostate.lpsa.asc()).show(5)

+------------+-----------+---+------------+---+------------+-------+-----+------------+
|      lcavol|    lweight|age|        lbph|svi|         lcp|gleason|pgg45|        lpsa|
+------------+-----------+---+------------+---+------------+-------+-----+------------+
|-0.579818495|2.769458829| 50|-1.386294361|  0|-1.386294361|      6|    0|-0.430782916|
|-0.994252273|3.319625728| 58|-1.386294361|  0|-1.386294361|      6|    0|-0.162518929|
|-0.510825624|2.691243083| 74|-1.386294361|  0|-1.386294361|      7|   20|-0.162518929|
|-1.203972804|3.282789151| 58|-1.386294361|  0|-1.386294361|      6|    0|-0.162518929|
| 0.751416089|3.432372999| 62|-1.386294361|  0|-1.386294361|      6|    0| 0.371563556|
+------------+-----------+---+------------+---+------------+-------+-----+------------+
only showing top 5 rows



* ascii  # ASCII of a string and replaces any non-ASCI characters with escape sequences
* asin   # arc sine
* atan   # arc tangent
* atan2  # arc tangent of two variables

## avg

In [26]:
# the average(mean) of the values in the lpsa column of prostate
prostate.select(avg(prostate.lpsa)).show()

+------------------+
|         avg(lpsa)|
+------------------+
|2.4783868787422683|
+------------------+



* base64      # encode binary type into base64 string
* bin         # convert a number into its binary string
* bitwiseNOT  # invert all the bits in a number
* broadcast   # optimizes join operations by broadcasting smaller DataFrames 
* bround      # round to the specified decimal point

## cbrt

In [27]:
# the cube root of the lpsa column of prostate
prostate.select('lpsa', cbrt(prostate.lpsa)).show(5)

+------------+-------------------+
|        lpsa|         CBRT(lpsa)|
+------------+-------------------+
|-0.430782916|-0.7552420410177275|
|-0.162518929|-0.5457176294010901|
|-0.162518929|-0.5457176294010901|
|-0.162518929|-0.5457176294010901|
| 0.371563556| 0.7189152621521183|
+------------+-------------------+
only showing top 5 rows



## ceil

In [28]:
# the ceiling of the lpsa column of prostate
prostate.select('lpsa', ceil(prostate.lpsa)).show(5)

+------------+----------+
|        lpsa|CEIL(lpsa)|
+------------+----------+
|-0.430782916|         0|
|-0.162518929|         0|
|-0.162518929|         0|
|-0.162518929|         0|
| 0.371563556|         1|
+------------+----------+
only showing top 5 rows



## coalesce
Return the first column that is not null.

In [29]:
df = spark.createDataFrame([(None, None), (1, None), (None, 2)], ('a', 'b'))
df.show()

+----+----+
|   a|   b|
+----+----+
|NULL|NULL|
|   1|NULL|
|NULL|   2|
+----+----+



In [30]:
df.select(coalesce(df.a, df.b)).show()

+--------------+
|coalesce(a, b)|
+--------------+
|          NULL|
|             1|
|             2|
+--------------+



## col
Returns a **Column** based on the given column name. 
It can save your some typing when the dataframe is very long.

In [31]:
prostate.show(5)

+------------+-----------+---+------------+---+------------+-------+-----+------------+
|      lcavol|    lweight|age|        lbph|svi|         lcp|gleason|pgg45|        lpsa|
+------------+-----------+---+------------+---+------------+-------+-----+------------+
|-0.579818495|2.769458829| 50|-1.386294361|  0|-1.386294361|      6|    0|-0.430782916|
|-0.994252273|3.319625728| 58|-1.386294361|  0|-1.386294361|      6|    0|-0.162518929|
|-0.510825624|2.691243083| 74|-1.386294361|  0|-1.386294361|      7|   20|-0.162518929|
|-1.203972804|3.282789151| 58|-1.386294361|  0|-1.386294361|      6|    0|-0.162518929|
| 0.751416089|3.432372999| 62|-1.386294361|  0|-1.386294361|      6|    0| 0.371563556|
+------------+-----------+---+------------+---+------------+-------+-----+------------+
only showing top 5 rows



In [32]:
prostate.select(col('lcavol'), col('age')).show(5)

+------------+---+
|      lcavol|age|
+------------+---+
|-0.579818495| 50|
|-0.994252273| 58|
|-0.510825624| 74|
|-1.203972804| 58|
| 0.751416089| 62|
+------------+---+
only showing top 5 rows



## collect_list

In [34]:
pdf = pd.DataFrame({
    'x': [1,2,2,3,4,4,4,4]
})
df = spark.createDataFrame(pdf)
df.show()

+---+
|  x|
+---+
|  1|
|  2|
|  2|
|  3|
|  4|
|  4|
|  4|
|  4|
+---+



In [35]:
# a list of all the values of x column
df.select(collect_list(df.x)).show()

+--------------------+
|     collect_list(x)|
+--------------------+
|[1, 2, 2, 3, 4, 4...|
+--------------------+



## collect_set

In [36]:
# a list of all the distinct values of x column
df.select(collect_set(df.x)).show()

+--------------+
|collect_set(x)|
+--------------+
|  [1, 2, 3, 4]|
+--------------+



## concat

In [37]:
df = spark.createDataFrame([['a', '1'], ['b', '2']], ['x', 'v'])
df.show()

+---+---+
|  x|  v|
+---+---+
|  a|  1|
|  b|  2|
+---+---+



In [38]:
# concatenate columns x and v into a new column
df.select('x', 'v', concat(df.x, df.v).alias('concate(x,v)')).show()

+---+---+------------+
|  x|  v|concate(x,v)|
+---+---+------------+
|  a|  1|          a1|
|  b|  2|          b2|
+---+---+------------+



## concat_ws

In [39]:
# concatenate columns x and v using '_' as a separator into a new column
df.select('x', 'v', concat_ws('_', df.x, df.v).alias('concate(x,v)')).show()

+---+---+------------+
|  x|  v|concate(x,v)|
+---+---+------------+
|  a|  1|         a_1|
|  b|  2|         b_2|
+---+---+------------+



## conv
convert a number from one base(radix) to another(like binary, decimal, hexadecimal)

## corr

In [40]:
prostate.show(5)

+------------+-----------+---+------------+---+------------+-------+-----+------------+
|      lcavol|    lweight|age|        lbph|svi|         lcp|gleason|pgg45|        lpsa|
+------------+-----------+---+------------+---+------------+-------+-----+------------+
|-0.579818495|2.769458829| 50|-1.386294361|  0|-1.386294361|      6|    0|-0.430782916|
|-0.994252273|3.319625728| 58|-1.386294361|  0|-1.386294361|      6|    0|-0.162518929|
|-0.510825624|2.691243083| 74|-1.386294361|  0|-1.386294361|      7|   20|-0.162518929|
|-1.203972804|3.282789151| 58|-1.386294361|  0|-1.386294361|      6|    0|-0.162518929|
| 0.751416089|3.432372999| 62|-1.386294361|  0|-1.386294361|      6|    0| 0.371563556|
+------------+-----------+---+------------+---+------------+-------+-----+------------+
only showing top 5 rows



In [41]:
# pearson correlation coefficient between column age and lpsa
prostate.select(corr(prostate.age, prostate.lpsa)).show(5)

+-------------------+
|    corr(age, lpsa)|
+-------------------+
|0.16959284228582772|
+-------------------+



## cos
calculate the cosine of an angle(in radians)

## cosh
compute the hyperbolic cosine of a number: $\cosh(x) = \frac{e^x + e^{-x}}{2}$

## count
number of non-null rows in a column

In [44]:
prostate.select(count(prostate.lpsa)).show()

+-----------+
|count(lpsa)|
+-----------+
|         97|
+-----------+



## countDistinct

In [45]:
iris.select(count(iris.species)).show()

+--------------+
|count(species)|
+--------------+
|           150|
+--------------+



## covar_pop
**population covariance: $\frac{1}{n}\sum_{i=1}^n(x_{i} - \bar{x})(y_{i} - \bar{y})$

In [46]:
prostate.select(covar_pop(prostate.age, prostate.lpsa)).show()

+--------------------+
|covar_pop(age, lpsa)|
+--------------------+
|  1.4424746293984458|
+--------------------+



## covar_samp
**sample covariance:  $\frac{1}{n-1}\sum_{i=1}^n(x_{i} - \bar{x})(y_{i} - \bar{y})$

In [47]:
prostate.select(covar_samp(prostate.age, prostate.lpsa)).show()

+---------------------+
|covar_samp(age, lpsa)|
+---------------------+
|   1.4575004067880128|
+---------------------+



## create_map

In [48]:
iris.show(5)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [49]:
# map using 'species' as the key and 'sepal_length' as the value
df = iris.select(create_map('species', 'sepal_length'))
df.show(5)

+--------------------------+
|map(species, sepal_length)|
+--------------------------+
|           {setosa -> 5.1}|
|           {setosa -> 4.9}|
|           {setosa -> 4.7}|
|           {setosa -> 4.6}|
|           {setosa -> 5.0}|
+--------------------------+
only showing top 5 rows



In [50]:
df.dtypes

[('map(species, sepal_length)', 'map<string,double>')]

## cume_dist
calculate the cumulative distribution of a column's values
It returns the realative rank of a value in a dataset as a number between 0 and 1

## current_date

In [51]:
df = spark.createDataFrame([[1],[2],[3],[4]], ['x'])
df.show()

+---+
|  x|
+---+
|  1|
|  2|
|  3|
|  4|
+---+



In [52]:
df.select('x', current_date()).show()

+---+--------------+
|  x|current_date()|
+---+--------------+
|  1|    2025-02-28|
|  2|    2025-02-28|
|  3|    2025-02-28|
|  4|    2025-02-28|
+---+--------------+



## current_tmestamp

In [53]:
df.select('x', current_timestamp()).show(truncate=False)

+---+-----------------------+
|x  |current_timestamp()    |
+---+-----------------------+
|1  |2025-02-28 03:06:53.923|
|2  |2025-02-28 03:06:53.923|
|3  |2025-02-28 03:06:53.923|
|4  |2025-02-28 03:06:53.923|
+---+-----------------------+



## date_add

In [54]:
df2 = df.select('x', current_date().alias('current_date'))
df2.show(5)

+---+------------+
|  x|current_date|
+---+------------+
|  1|  2025-02-28|
|  2|  2025-02-28|
|  3|  2025-02-28|
|  4|  2025-02-28|
+---+------------+



In [55]:
# add 10 days to the current date
df2.select('x', 'current_date', date_add(df2.current_date, 10)).show()

+---+------------+--------------------------+
|  x|current_date|date_add(current_date, 10)|
+---+------------+--------------------------+
|  1|  2025-02-28|                2025-03-10|
|  2|  2025-02-28|                2025-03-10|
|  3|  2025-02-28|                2025-03-10|
|  4|  2025-02-28|                2025-03-10|
+---+------------+--------------------------+



## date_format

In [56]:
df2.select('x', 'current_date', date_format('current_date', 'MM/dd/yyyy').alias('new_date')).show()

+---+------------+----------+
|  x|current_date|  new_date|
+---+------------+----------+
|  1|  2025-02-28|02/28/2025|
|  2|  2025-02-28|02/28/2025|
|  3|  2025-02-28|02/28/2025|
|  4|  2025-02-28|02/28/2025|
+---+------------+----------+

